In [7]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from selenium.webdriver.common.alert import Alert
from selenium import webdriver
from multiprocessing import Pool
import time

# 시트에 접근허용
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1FBm8hL-ujJy6ewY-Gi4atPl4SgD-eRv4H3RFgVDFDLs'
print("동의건 : 1, 비동의건 : 2")
select_range = input()
if select_range == "1" :
    SAMPLE_RANGE_NAME = 'VR 공실_매물등록(VR동의건)!'
elif select_range == "2" : 
    SAMPLE_RANGE_NAME = '일반 공실_매물등록(VR비동의건 전부)!'
else :
    print("1 혹은 2를 입력해주세요.")
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
         creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '매등접근.json', SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()



# 작업할 행 선택
selectRowStart = input(int)
selectRowEnd = input(int)

allData = int(selectRowEnd) - int(selectRowStart) + 1
success = 0

driver = webdriver.Chrome("C:\\Users\kwh\Desktop/ppp/chromedriver")
driver.get('https://admin.zigbang.com/')
elem = driver.find_element_by_id("UserName")
elem.send_keys("bizsupport230@zigbang.com")
elem = driver.find_element_by_id("Password") 
elem.send_keys("fifa2002")
elem.submit()
while selectRowEnd >= selectRowStart :
    # 예외변수
    throwData = False
    throwText = ""
    # 데이터가져오기
    list1 = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'I' + selectRowStart).execute()
    list1 = list1.get("values", [])[0][0].split("/")
    if (len(list1) < 6) : list1.append("")
    # 106제곱/매매18500/주인거주/바로가능/남동향/가겨조정가능, 사용많이안해서 상태좋습니다.
    # 0 : 평수 1 : 가격 2 : x 3 : 입주가능일 4 : 방향 5 : 한줄설명
    target_apart = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'F' + selectRowStart).execute()
    target_apart = target_apart.get("values", [])[0][0]
    rent = 0
    if ((list1[1].find("매매") != -1 and list1[1].find("전세") != -1) or (list1[1].find("매매") != -1 and list1[1].find("월세") != -1) or (list1[1].find("월세") != -1 and list1[1].find("전세") != -1)):
        throwData = True
        throwText = throwText + "가격"
        print("!")
    elif list1[1].find("매매") != -1 : 
        sales_type = "매매"
        deposit = list1[1].replace("매매", "")
    elif list1[1].find("전세") != -1 :
        sales_type = "전세"
        deposit = list1[1].replace("전세", "")
    elif list1[1].find("월세") != -1 :
        sales_type = "월세"
        prize = list1[1].replace("월세", "")
        prize = prize.replace("-", " ")
        prize = prize.split(" ")
        useRent = 0
        for void in prize:
            if isNumber(void) == True :
                if useRent == 0 : 
                    deposit = void
                else : rent = void
                useRent += 1
        print(deposit + " " + rent)
    room_type = list1[0].replace(" ", "")
    name = '공실매물화'
    number = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'H' + selectRowStart).execute()
    number = number.get("values")[0][0]
    if len(number) == 11 :
        number1 = number[0:3]
        number2 = number[3:7]
        number3 = number[7:11]
    else :
        number1 = number[0:3]
        number2 = number[3:6]
        number3 = number[6:10]
        
    def isNumber(s):
            try:
                float(s)
                return True
            except ValueError:
                return False
            
    dongho = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'G' + selectRowStart).execute()
    dongho = dongho.get("values", [])[0][0].split("/")
    dong = dongho[0].replace(" ", "")
    ho = dongho[1].replace("호", "")
    if isNumber(ho) == False :
        floor = 1
        throwData = True
        throwText = "호수확인필요"
    else : 
        floor = str(round(int(ho) / 100))
    direction = list1[4].replace("향", "")
    direction = direction.replace(" ", "")
    date = list1[3]
    title = list1[5]
    
    #부동산 개수 파악 
    e = 1
    e2 = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'Q' + selectRowStart).execute()
    if len(e2) > 2 :
        e2 = e2.get("values")[0][0]
        e = e + 1
    e3 = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'U' + selectRowStart).execute()
    if len(e3) > 2 :
        e3 = e3.get("values")[0][0]
        e = e + 1
    
    # 크롬브라우저 실행
    n = 1
    while e >= n :
        
        
        
        if n == 1 :
            adminCode = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'Z' + selectRowStart).execute()
            adminCode = adminCode.get("values", [])[0][0]
        elif n == 2 :
            adminCode = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'AB' + selectRowStart).execute()
            adminCode = adminCode.get("values", [])[0][0]
        elif n == 3 :
            adminCode = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'AD' + selectRowStart).execute()
            adminCode = adminCode.get("values", [])[0][0]
            
        driver.get(adminCode)
    
        # 아파트 단지명 선택
        select = driver.find_element_by_id("target_apart")
        allOptions = select.find_elements_by_tag_name("option")
        for option in allOptions:
            adress = option.get_attribute("data-name")
            if adress == target_apart :
                option.click()
        # 직접확보매물 체크
        elem = driver.find_element_by_id("item-exclusive")
        elem.click()
        #거래유형 선택
        select = driver.find_element_by_id("sales_type")
        allOptions = select.find_elements_by_tag_name("option")
        for option in allOptions:
            adress = option.get_attribute("value")
            if adress == sales_type :
                option.click()

        #가격 입력
        elem = driver.find_element_by_id("deposit") 
        elem.send_keys(deposit)
        if rent != 0 :
            elem = driver.find_element_by_id("rent") 
            elem.send_keys(rent)    
        #면젹유형 선택
        if room_type.find("평") != -1 or room_type.find("제곱") :
            select = driver.find_element_by_id("room_type")
            allOptions = select.find_elements_by_tag_name("option")
            for option in allOptions:
                adress = option.text
                if adress == room_type :
                    option.click()
        else : 
            throwText = throwText + "면적유형"
            select = driver.find_element_by_id("room_type")
            allOptions = select.find_elements_by_tag_name("option")
            for option in allOptions:
                adress = option.text
                if adress == room_type :
                    option.click()
    
        
        #소유주
        elem = driver.find_element_by_id("ex_owner_name") 
        elem.send_keys(name)
       
        #연락처
        select = driver.find_element_by_id("ex_owner_phone_1")
        allOptions = select.find_elements_by_tag_name("option")
        for option in allOptions:
            adress = option.get_attribute("value")
            if adress == number1 :
                option.click()
        elem = driver.find_element_by_id("ex_owner_phone_2") 
        elem.send_keys(number2)
        elem = driver.find_element_by_id("ex_owner_phone_3") 
        elem.send_keys(number3)
        
        #동
        select = driver.find_element_by_id("ex_dong")
        allOptions = select.find_elements_by_tag_name("option")
        for option in allOptions:
            adress = option.text
            if adress == dong :
                option.click()
        #호수::

        if isNumber(ho) == True :
            elem = driver.find_element_by_id("ex_ho") 
            elem.send_keys(ho)
            driver.implicitly_wait(100)
        #층
            select = driver.find_element_by_id("ex_item_floor")
            allOptions = select.find_elements_by_tag_name("option")
            for option in allOptions:
                adress = option.get_attribute("value")
                if adress == floor :
                    option.click()  
        else : throwText = throwText + "호수"
 
        #방향
        select = driver.find_element_by_id("ex_direction")
        allOptions = select.find_elements_by_tag_name("option")
        for option in allOptions:
            adress = option.get_attribute("value")
            if adress == direction :
                option.click()
    
        #입주가능일
        elem = driver.find_element_by_id("movein_date") 
        elem.send_keys(date)
   
        #한줄설명
        titlecopy = title
        if len(titlecopy.replace(" ", "")) >= 7 and len(titlecopy.replace(" ", "")) < 33  :
            elem = driver.find_element_by_id("title") 
            elem.send_keys(title)
        elif len(titlecopy.replace(" ", "")) == 0 :
            elem = driver.find_element_by_id("title") 
            elem.send_keys(target_apart + "아파트입니다.")
        else : 
            elem = driver.find_element_by_id("title") 
            elem.send_keys("한줄설명수정해주세요.")
            throwData = True
            throwText = throwText + "한줄설명길이"
            
        n = n + 1

        driver.execute_script("validateSaveItem()")
        driver.execute_script("getSaveParam()")
        
        driver.execute_script("getUploader().init()")
        #저장

       
        
        
"""
        code = driver.page_source.replace("https://admin.zigbang.com/apartitem/ItemInfo?id=", "")
        requestCode = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'E' + selectRowStart).execute()
        requestCode = requestCode.get("values", [])[0][0]
        print(requestCode + "완료")

        if throwData == False :
            
            elem = driver.find_element_by_id("checkButton1") 
            da = Alert(driver)
            da.accept()
           

            success = success + 1
            
             
        else : 
            sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'AF' + selectRowStart, body = throwText).execute()
       
    
        
        if i == 0 :
            sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'AA' + selectRowStart, body = code).execute()
        elif i == 1 :
            sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'AC' + selectRowStart, body = code).execute()
        elif i == 2 :
            sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME + 'AE' + selectRowStart, body = code).execute()
    
    selectRowStart = str(int(selectRowStart) + 1)
print("전체 데이터 : " + allData + ", " + "완료 : " + success)
"""

동의건 : 1, 비동의건 : 2
2
<class 'int'>14870
<class 'int'>14870


KeyboardInterrupt: 